In [9]:
import pandas as pd
import joblib
import time

from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV  # Bayesian optimization: utilizado para optimizar hiperparámetros

import xgboost  as xgb

from Funcoes_Comuns import avaliar_modelo, registrar_modelo

Recuperar base já pré-processada

In [2]:
# Obter dados
df_enem = pd.read_pickle('Bases\MICRODADOS_ENEM_2023_tratados.pkl')

In [3]:
variaveis_alvo = ['NUM_NOTA_MT', 'NUM_NOTA_LC', 'NUM_NOTA_CN', 'NUM_NOTA_CH', 'NUM_NOTA_REDACAO']

# separar em treino e teste
X = df_enem.drop(columns=variaveis_alvo)
y = df_enem[variaveis_alvo]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Ajuste de tipo para MLflow
# Converter colunas inteiras para float
X_train = X_train.astype({col: 'float' for col in X_train.select_dtypes('int').columns})
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

Modelo base

In [65]:
# Treinar modelo XGBoost
modelo_xgb = xgb.XGBRegressor(n_estimators=200,
                              max_depth=3,
                              learning_rate=0.5,
                              colsample_bytree=0.5,
                              objective='reg:absoluteerror',
                              enable_categorical=True,
                              reg_alpha=0.3,
                              reg_lambda=0.7,
                              random_state=42,
                              n_jobs=-1)

start_time = time.time()

modelo_xgb.fit(X_train, 
                y_train['NUM_NOTA_CH'], 
                eval_set=[(X_test, y_test['NUM_NOTA_CH'])])

tempo_treino = time.time() - start_time

[0]	validation_0-mae:61.42720
[1]	validation_0-mae:59.21703
[2]	validation_0-mae:57.63742
[3]	validation_0-mae:57.02697
[4]	validation_0-mae:56.72796
[5]	validation_0-mae:56.53725
[6]	validation_0-mae:56.38173
[7]	validation_0-mae:56.30748
[8]	validation_0-mae:56.26206
[9]	validation_0-mae:56.18912
[10]	validation_0-mae:56.14793
[11]	validation_0-mae:56.08672
[12]	validation_0-mae:56.04974
[13]	validation_0-mae:56.01596
[14]	validation_0-mae:55.95746
[15]	validation_0-mae:55.94932
[16]	validation_0-mae:55.95103
[17]	validation_0-mae:55.94178
[18]	validation_0-mae:55.93887
[19]	validation_0-mae:55.93680
[20]	validation_0-mae:55.93290
[21]	validation_0-mae:55.90855
[22]	validation_0-mae:55.88307
[23]	validation_0-mae:55.87902
[24]	validation_0-mae:55.87598
[25]	validation_0-mae:55.86850
[26]	validation_0-mae:55.86113
[27]	validation_0-mae:55.85284
[28]	validation_0-mae:55.85132
[29]	validation_0-mae:55.84923
[30]	validation_0-mae:55.84516
[31]	validation_0-mae:55.82831
[32]	validation_0-

In [66]:
# Previsões
y_pred = modelo_xgb.predict(X_test)

In [67]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento,
                 parametros={**modelo_xgb.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                 X_train=X_train,
                 y_train=y_train,
                 y_test=y_test,
                 y_pred=y_pred,
                 variavel_alvo='NUM_NOTA_CH',
                 modelo=modelo_xgb,
                 nome_modelo='modelo_xgbm_base',
                 descricao_modelo='Modelo XGBRegressor base')

d:\Armazenamento\MBA\TCC\TccMbaUspEsalq\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Registered model 'modelo_xgbm_base' already exists. Creating a new version of this model...
2025/05/21 21:18:44 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation.

🏃 View run capricious-swan-988 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/7cc8d4bbcd9f40758e3645ebc4279f74
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_xgbm_base
Rastreamento do MLflow finalizado.


Created version '12' of model 'modelo_xgbm_base'.


In [68]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_xgb.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred, "teste")

MAE (treino): 54.35
RMSE (treino): 69.96
R2 (treino): 0.32
MAE (teste): 55.27
RMSE (teste): 70.78
R2 (teste): 0.30


In [117]:
# Salvar o modelo otimizado como um arquivo pickle
joblib.dump(modelo_xgb, 'Modelos\modelo_xgb_base.pkl')

['Modelos\\modelo_xgb_base.pkl']

Bayes Search

In [184]:
modelo_xgb_bayes = xgb.XGBRegressor(enable_categorical=True, 
                                    eval_metric=['rmse', 'mae'],
                                    objective='reg:absoluteerror',
                                    early_stopping_rounds=200)

In [185]:
param_grid = {
    'n_estimators': [300, 350, 600],
    'max_depth': [2, 3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.5, 1],
    'colsample_bytree': [0.3, 0.5, 0.7],
    'reg_alpha': [0.05, 0.1, 0.3, 0.5],
    'reg_lambda': [0.3, 0.5, 0.7]
}

In [186]:
bayes_search = BayesSearchCV(
    estimator=modelo_xgb_bayes,        # Modelo a ser otimizado
    search_spaces=param_grid,          # Espaço de busca definido acima
    scoring='neg_mean_absolute_error', # Critério de seleção
    n_iter=20,                         # Número de avaliações do modelo
    cv=3,                              # Validação cruzada
    random_state=42,                   # Semente para reprodutibilidade
    n_jobs=-1,                         # Paralelização total dos cálculos
    verbose=1                          # 0 = sem mensagens, 1 = mensagens de progresso, 2 = mensagens detalhadas
)

In [187]:
# Criar Eval Set para validação cruzada (15% do conjunto de treino)
X_train_bayes, X_eval, y_train_bayes, y_eval = train_test_split(
    X_train,
    y_train['NUM_NOTA_CH'],
    test_size=0.15,
    random_state=42
)

In [188]:
fit_params = {
    'eval_set': [(X_eval, y_eval)],   # Conjunto de validação
    'verbose': 1,                     # Mensagens durante o ajuste
}

In [189]:
# Executar a busca Bayesiana

start_time = time.time()
bayes_search.fit(X_train_bayes, y_train_bayes, **fit_params) 

# Parar o cronômetro
end_time = time.time()
elapsed_time = end_time - start_time

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

In [190]:
# Resultados da busca Bayesiana

print("Melhores parâmetros encontrados:")
print(bayes_search.best_params_)
print(f"Tempo total de execução: {elapsed_time:.2f} segundos")

Melhores parâmetros encontrados:
OrderedDict([('colsample_bytree', 0.3), ('learning_rate', 0.1), ('max_depth', 7), ('n_estimators', 350), ('reg_alpha', 0.5), ('reg_lambda', 0.3)])
Tempo total de execução: 2789.40 segundos


In [191]:
# Treinar o modelo com os melhores parâmetros encontrados
modelo_xgb_bayes.set_params(**bayes_search.best_params_)

start_time = time.time()

# Treinamento do modelo com os melhores parâmetros encontrados
modelo_xgb_bayes.fit(X_train_bayes, 
                        y_train_bayes, 
                        eval_set=[(X_eval, y_eval)])

tempo_treino = time.time() - start_time

[0]	validation_0-rmse:82.98350	validation_0-mae:66.25879
[1]	validation_0-rmse:81.24635	validation_0-mae:64.75727
[2]	validation_0-rmse:79.98585	validation_0-mae:63.66403
[3]	validation_0-rmse:78.85288	validation_0-mae:62.68093
[4]	validation_0-rmse:78.00229	validation_0-mae:61.93376
[5]	validation_0-rmse:77.17143	validation_0-mae:61.20620
[6]	validation_0-rmse:76.29227	validation_0-mae:60.42843
[7]	validation_0-rmse:75.71997	validation_0-mae:59.92023
[8]	validation_0-rmse:75.09989	validation_0-mae:59.36603
[9]	validation_0-rmse:74.64993	validation_0-mae:58.95829
[10]	validation_0-rmse:74.14946	validation_0-mae:58.51801
[11]	validation_0-rmse:73.66999	validation_0-mae:58.08941
[12]	validation_0-rmse:73.28934	validation_0-mae:57.74516
[13]	validation_0-rmse:73.08668	validation_0-mae:57.55142
[14]	validation_0-rmse:72.87868	validation_0-mae:57.36043
[15]	validation_0-rmse:72.75870	validation_0-mae:57.25133
[16]	validation_0-rmse:72.64014	validation_0-mae:57.14074
[17]	validation_0-rmse:7

In [192]:
# Previsões
y_pred_bayes = modelo_xgb_bayes.predict(X_test)

In [193]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento,
                    modelo=modelo_xgb_bayes,
                    parametros={**modelo_xgb_bayes.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                    X_train=X_train,
                    y_train=y_train,
                    y_test=y_test,
                    y_pred=y_pred_bayes,
                    variavel_alvo='NUM_NOTA_CH',
                    nome_modelo='modelo_xgb_bayes',
                    descricao_modelo='Modelo XGBMRegressor otimizado com BayesSearchCV')

d:\Armazenamento\MBA\TCC\TccMbaUspEsalq\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Registered model 'modelo_xgb_bayes' already exists. Creating a new version of this model...
2025/05/21 23:20:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation.

🏃 View run defiant-crow-79 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/3053a8480abb462f94d6b99efb9a9fbe
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_xgb_bayes
Rastreamento do MLflow finalizado.


Created version '6' of model 'modelo_xgb_bayes'.


In [194]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_xgb_bayes.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_bayes, "teste")

MAE (treino): 53.07
RMSE (treino): 68.74
R2 (treino): 0.34
MAE (teste): 54.97
RMSE (teste): 70.43
R2 (teste): 0.31


In [195]:
# Salvar o modelo otimizado como um arquivo pickle
joblib.dump(modelo_xgb_bayes, 'Modelos\modelo_xgb_bayes.pkl')

['Modelos\\modelo_xgb_bayes.pkl']